In [221]:
# import library to solve the problem
import numpy as np
import pandas as pd
import gamspy as gp
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

# Problem 1: 
Produce n products satisfying production <= demand 
Use the 2-SLPWR when n = 8 products, the number of scenarios S = 2 with density ps = 1/2, the number of parts to be ordered before production
m = 5, we randomly simulate data vector b, l, q, s and matrix A of size n × m.


Assume that the random demand vector w = D = (D1, D2, ..., Dn) where each wi with density pi follows the binomial distribution Bin(10, 1/2)

# Firstly, we need to create some random array and some data to use in problem1


In [222]:
n, m, nofScen = 8, 5, 2 # num of products, number of parts to be ordered before production and num of scenario

In [223]:
# create name of each elements in array
products = ['product ' + str(i) for i in range(1, n+1)]
parts = ['parts ' + str(j) for j in range(1, m+1)]
scen = ['scenario ' + str(k) for k in range(1, nofScen+1)]

In [224]:
products

['product 1',
 'product 2',
 'product 3',
 'product 4',
 'product 5',
 'product 6',
 'product 7',
 'product 8']

In [225]:
parts

['parts 1', 'parts 2', 'parts 3', 'parts 4', 'parts 5']

In [226]:
scen

['scenario 1', 'scenario 2']

In [227]:
# each scenario with density = 0.5
density = pd.DataFrame(
    {
        'scenarios' : scen,
        'density' : [0.5, 0.5]
    }
).set_index('scenarios')
density

,density
scenarios,
scenario 1,0.5
scenario 2,0.5


## Create some random matrix as requirement

In [228]:
# use numpy to random
np.random.seed(89)
# first stage random matrix
# demands: each product is assign with 2 scen

demands = pd.DataFrame({
    'products' : np.repeat(products, nofScen),
    'scenarios' : scen * n,
    'demands' : np.array([np.random.binomial(10, 0.5, n) for i in range(2)]).reshape(-1)
}).set_index(['products',  'scenarios'])

# l : additionally cost to sasitfy a unit demand of product i

l = pd.DataFrame({
    'products' : products,
    'additionally_cost': np.random.randint(0, 300, n)  
}).set_index('products')

# q : selling price of each product i

q = pd.DataFrame({
    'products' : products,
    'selling_price' : np.random.randint(50, 300, n)
}).set_index('products')

# c : cost coefficients

c = pd.DataFrame({
    'products': products,
    'cost_coefficients' : l['additionally_cost'] - q['selling_price']
}).set_index('products')

# b : preorder cost of each part j (before the demand know)
b = pd.DataFrame({
    'parts' : parts, 
    'preorder _cost_each_part': np.random.randint(0, 20, m)
}).set_index('parts')
# s : assessed salvage values sj

s = pd.DataFrame({
    'parts': parts,
    'assessed_salvage_values_each_part' : np.random.randint(0, 10, m)
}).set_index('parts')

# condition : sj < bj with j in range(m)
for j in range(m):
    if s.iloc[j, 0] >= b.iloc[j, 0]:
        s.iloc[j, 0] = b.iloc[j, 0] - 10

# A: each product i requires aij units of part j
requirements = pd.DataFrame({
    'products' : np.repeat(products, m),
    'parts': parts*n,
    'units required ': np.array([np.random.randint(0, 10) for i in range(n) for j in range(m)]).reshape(-1)
}).set_index(['products', 'parts'])

In [229]:
print('demands\n', demands)

demands
                       demands
products  scenarios          
product 1 scenario 1        5
          scenario 2        4
product 2 scenario 1        4
          scenario 2        3
product 3 scenario 1        5
          scenario 2        2
product 4 scenario 1        3
          scenario 2        5
product 5 scenario 1        3
          scenario 2        5
product 6 scenario 1        7
          scenario 2        4
product 7 scenario 1        3
          scenario 2        6
product 8 scenario 1        3
          scenario 2        5


In [230]:
print('Additionally cost to sasitfy a unit demand of product i\n', l)

Additionally cost to sasitfy a unit demand of product i
            additionally_cost
products                    
product 1                173
product 2                287
product 3                275
product 4                 93
product 5                 40
product 6                240
product 7                 31
product 8                285


In [231]:
print('Selling price of each product i\n', q)

Selling price of each product i
            selling_price
products                
product 1             66
product 2            205
product 3            191
product 4            288
product 5            236
product 6            160
product 7            201
product 8            255


In [232]:
print('Cost coefficients\n', c)

Cost coefficients
            cost_coefficients
products                    
product 1                107
product 2                 82
product 3                 84
product 4               -195
product 5               -196
product 6                 80
product 7               -170
product 8                 30


In [233]:
print('Preorder cost of each part j (before the demand know) \n', b)

Preorder cost of each part j (before the demand know) 
          preorder _cost_each_part
parts                            
parts 1                         1
parts 2                        18
parts 3                         9
parts 4                         4
parts 5                        18


In [234]:
print('Assessed salvage values sj \n', s)

Assessed salvage values sj 
          assessed_salvage_values_each_part
parts                                     
parts 1                                 -9
parts 2                                  9
parts 3                                  2
parts 4                                 -6
parts 5                                  5


In [235]:
print('Each product i requires aij units of part j\n', requirements)

Each product i requires aij units of part j
                    units required 
products  parts                   
product 1 parts 1                1
          parts 2                3
          parts 3                9
          parts 4                0
          parts 5                2
product 2 parts 1                7
          parts 2                2
          parts 3                3
          parts 4                0
          parts 5                1
product 3 parts 1                1
          parts 2                6
          parts 3                1
          parts 4                4
          parts 5                5
product 4 parts 1                2
          parts 2                8
          parts 3                6
          parts 4                9
          parts 5                4
product 5 parts 1                6
          parts 2                2
          parts 3                5
          parts 4                7
          parts 5                1
product 6 

# Secondly, building model to solve the problem 

In [236]:
# create container to include set and expression
modelp1 = Container()

In [237]:
#create set to push into container
i = Set(container= modelp1, name = 'i', description=  'numbers of products', records = products) # i ~ products
j = Set(container= modelp1, name = 'j', description=  'number of parts', records = parts) # j ~ parts
k = Set(container= modelp1, name = 'k', description= 'number of scenarios', records= scen) # k ~ scenarios

## Create parameter to add into model

In [238]:
# add A as requirements each products require aij per each parts 
A = Parameter(container = modelp1, name = 'A', description= 'Requirements of each product i requires aij part j', domain=[i, j], records = requirements.reset_index())
A.records

,products,parts,value
0,product 1,parts 1,1.0
1,product 1,parts 2,3.0
2,product 1,parts 3,9.0
3,product 1,parts 4,0.0
4,product 1,parts 5,2.0
5,product 2,parts 1,7.0
6,product 2,parts 2,2.0
7,product 2,parts 3,3.0
8,product 2,parts 4,0.0
9,product 2,parts 5,1.0


In [239]:
d = Parameter(container = modelp1, name = 'd', domain=[i, k], description = 'Demand of each product with each scenario', records=demands.reset_index())
d.records

,products,scenarios,value
0,product 1,scenario 1,5.0
1,product 1,scenario 2,4.0
2,product 2,scenario 1,4.0
3,product 2,scenario 2,3.0
4,product 3,scenario 1,5.0
5,product 3,scenario 2,2.0
6,product 4,scenario 1,3.0
7,product 4,scenario 2,5.0
8,product 5,scenario 1,3.0
9,product 5,scenario 2,5.0


In [240]:
l = Parameter(container = modelp1, name = 'l', domain = [i], description = 'cost coeffiecients per each product', records = l.reset_index())
l.records

,products,value
0,product 1,173.0
1,product 2,287.0
2,product 3,275.0
3,product 4,93.0
4,product 5,40.0
5,product 6,240.0
6,product 7,31.0
7,product 8,285.0


In [241]:
q = Parameter(container = modelp1, name = 'q', domain = [i], description = 'cost coeffiecients per each product', records = q.reset_index())
q.records

,products,value
0,product 1,66.0
1,product 2,205.0
2,product 3,191.0
3,product 4,288.0
4,product 5,236.0
5,product 6,160.0
6,product 7,201.0
7,product 8,255.0


In [242]:
c = Parameter(container = modelp1, name = 'c', domain = [i], description = 'cost coeffiecients per each product', records = c.reset_index())
c.records

,products,value
0,product 1,107.0
1,product 2,82.0
2,product 3,84.0
3,product 4,-195.0
4,product 5,-196.0
5,product 6,80.0
6,product 7,-170.0
7,product 8,30.0


In [243]:
b = Parameter(container = modelp1, name = 'b', domain = [j], description = 'Preorder cost of each part j (before the demand know)', records = b.reset_index())
b.records

,parts,value
0,parts 1,1.0
1,parts 2,18.0
2,parts 3,9.0
3,parts 4,4.0
4,parts 5,18.0


In [244]:
s = Parameter(container = modelp1, name = 's', domain = [j], description = 'Assessed salvage values sj ', records = s.reset_index())
s.records

,parts,value
0,parts 1,-9.0
1,parts 2,9.0
2,parts 3,2.0
3,parts 4,-6.0
4,parts 5,5.0


In [245]:
prob = Parameter(container = modelp1, name = 'p', domain = [k], description = 'Density of each scenario', records = density.reset_index())
prob.records

,scenarios,value
0,scenario 1,0.5
1,scenario 2,0.5


## Then create decision variables to solve the problem

In [246]:
x = Variable(container = modelp1, name = 'x', type = 'INTEGER', domain = j, description = 'the numbers of parts to be ordered before production')
y = Variable(container = modelp1, name = 'y', type = 'INTEGER', domain = [j, k], description = 'the number of parts left in inventory') 
z = Variable(container = modelp1, name = 'z', type = 'INTEGER', domain = [i, k], description = 'the number of units produced')
# both y and z depend on relization d of random demand so that domain of y and z is [j, k] and [i, k]

## Create contrainst to solve the problem

In [247]:
c1 = Equation(container = modelp1, name = 'c1', domain = [j,k]) # y = x − AT*z
c1[j,k] = y[j,k] == x[j] - Sum(i, A[i, j]*z[i, k]) # y = x − AT*z

In [248]:
c2 = Equation(container = modelp1, name = 'c2', domain = j)
c2[j] = x[j] >= 0

In [249]:
c3 = Equation(container = modelp1, name = 'c3', domain =[j,k])
c3[j,k] = y[j,k] >= 0

In [250]:
c4 = Equation(container = modelp1, name = 'c4', domain= [i,k]) 
c4[i,k] = z[i,k] >= 0 

In [251]:
c5 = Equation(container = modelp1, name = 'c5', domain=[i,k])
c5[i, k] =z[i,k] <= d[i, k]  #zi ≤ di , i = 1, . . . , n

# Solve the problem

In [252]:
Z = Sum(i, c[i]*z[i, k] ) - Sum(j,  s[j]*y[j,k])
objfunction = Sum(j,b[j] * x[j]) + Sum(k, Z*prob[k])

In [253]:
modelp1.getEquations()

[<Eq Equation `c1` (0x211f0fc0cd0)>,
 <Eq Equation `c2` (0x211f0fc3ad0)>,
 <Eq Equation `c3` (0x211f0fc9410)>,
 <Eq Equation `c4` (0x211f0fc91d0)>,
 <Eq Equation `c5` (0x211f0fcd150)>]

In [254]:
problem1 = Model(container=modelp1, name = 'problem_1', equations = modelp1.getEquations(), problem = 'MIP', sense = Sense.MIN, objective = objfunction)

# Solution

In [255]:
problem1.solve()
problem1.objective_value

-342.0

In [256]:
print("The numbers of parts to be ordered before production ")
x.records.set_index("j")

The numbers of parts to be ordered before production 


,level,marginal,lower,upper,scale
j,,,,,
parts 1,39.0,1.0,0.0,inf,1.0
parts 2,18.0,18.0,0.0,inf,1.0
parts 3,27.0,9.0,0.0,inf,1.0
parts 4,24.0,4.0,0.0,inf,1.0
parts 5,3.0,18.0,0.0,inf,1.0


In [257]:
print("The number of parts left in inventory ")
y.records.set_index(["j", "k"])

The number of parts left in inventory 


level  marginal  lower  upper  scale
j       k                                               
parts 1 scenario 1    0.0       4.5    0.0    inf    1.0
        scenario 2    0.0       4.5    0.0    inf    1.0
parts 2 scenario 1    0.0      -4.5    0.0    inf    1.0
        scenario 2    0.0      -4.5    0.0    inf    1.0
parts 3 scenario 1    0.0      -1.0    0.0    inf    1.0
        scenario 2    0.0      -1.0    0.0    inf    1.0
parts 4 scenario 1    0.0       3.0    0.0    inf    1.0
        scenario 2    0.0       3.0    0.0    inf    1.0
parts 5 scenario 1    0.0      -2.5    0.0    inf    1.0
        scenario 2    0.0      -2.5    0.0    inf    1.0

In [258]:
print("The number of units produced ")
z.records.set_index(["i","k"])

The number of units produced 


level  marginal  lower  upper  scale
i         k                                               
product 1 scenario 1    0.0      53.5    0.0    inf    1.0
          scenario 2    0.0      53.5    0.0    inf    1.0
product 2 scenario 1    0.0      41.0    0.0    inf    1.0
          scenario 2    0.0      41.0    0.0    inf    1.0
product 3 scenario 1    0.0      42.0    0.0    inf    1.0
          scenario 2    0.0      42.0    0.0    inf    1.0
product 4 scenario 1    0.0     -97.5    0.0    inf    1.0
          scenario 2    0.0     -97.5    0.0    inf    1.0
product 5 scenario 1    3.0     -98.0    0.0    inf    1.0
          scenario 2    3.0     -98.0    0.0    inf    1.0
product 6 scenario 1    0.0      40.0    0.0    inf    1.0
          scenario 2    0.0      40.0    0.0    inf    1.0
product 7 scenario 1    3.0     -85.0    0.0    inf    1.0
          scenario 2    3.0     -85.0    0.0    inf    1.0
product 8 scenario 1    0.0      15.0    0.0    inf    1.0
          scenario 2    0.0      15.0    0.0    inf    1.0